# Gene expression units and calculation


**RPM or CPM (Reads per million mapped reads or Counts per million mapped reads)**


RPM (also known as CPM) is a basic gene expression unit that normalizes only for sequencing depth (depth-normalized counts). The RPM is biased in some applications where the gene length influences gene expression, such as RNA-seq.



In [6]:
from bioinfokit.analys import norm, get_data
# load sugarcane RNA-seq expression dataset (Published in Bedre et al., 2019)
df = get_data('sc_exp').data
df.head(2)


Matplotlib is building the font cache; this may take a moment.


,gene,ctr1,ctr2,ctr3,trt1,trt2,trt3,length
0,Sobic.001G000200,338,324,246,291,202,168,1982.0
1,Sobic.001G000400,49,21,53,16,16,11,4769.0


In [7]:
#import sys
#!{sys.executable} -m pip install bioinfokit


In [8]:
# as this data has gene length column, we will drop length column
df = df.drop(['length'], axis=1)
# make gene column as index column
df = df.set_index('gene')
df.head(2)


,ctr1,ctr2,ctr3,trt1,trt2,trt3
gene,,,,,,
Sobic.001G000200,338,324,246,291,202,168
Sobic.001G000400,49,21,53,16,16,11


In [9]:
# now, normalize raw counts using CPM method 
nm = norm()
nm.cpm(df=df)
# get CPM normalized dataframe
cpm_df = nm.cpm_norm
cpm_df.head(2)


,ctr1,ctr2,ctr3,trt1,trt2,trt3
gene,,,,,,
Sobic.001G000200,100.695004,101.731189,74.721094,92.633828,74.270713,95.314714
Sobic.001G000400,14.597796,6.593688,16.098447,5.093269,5.882829,6.240844


**RPKM (Reads per kilo base of transcript per million mapped reads)**


RPKM (reads per kilobase of transcript per million reads mapped) is a gene expression unit that measures the expression levels (mRNA abundance) of genes or transcripts. RPKM is a gene length normalized expression unit that is used for identifying the differentially expressed genes by comparing the RPKM values between different experimental conditions. Generally, the higher the RPKM of a gene, the higher the expression of that gene.



RPKM/FPKM does not represent the accurate measure of relative RNA molar concentration (rmc) and can be biased towards identifying the differentially expressed genes as the total normalized counts for each sample will be different 3,4. TPM is proposed as an alternative to the RPKM.



In [10]:
from bioinfokit.analys import norm, get_data
# load sugarcane RNA-seq expression dataset (Published in Bedre et al., 2019)
df = get_data('sc_exp').data
df.head(2)


,gene,ctr1,ctr2,ctr3,trt1,trt2,trt3,length
0,Sobic.001G000200,338,324,246,291,202,168,1982.0
1,Sobic.001G000400,49,21,53,16,16,11,4769.0


In [11]:
# make gene column as index column
df = df.set_index('gene')
df.head(2)



,ctr1,ctr2,ctr3,trt1,trt2,trt3,length
gene,,,,,,,
Sobic.001G000200,338,324,246,291,202,168,1982.0
Sobic.001G000400,49,21,53,16,16,11,4769.0


In [12]:
# now, normalize raw counts using RPKM method
# gene length must be in bp
nm = norm()
nm.rpkm(df=df, gl='length')
# get RPKM normalized dataframe
rpkm_df = nm.rpkm_norm
rpkm_df.head(2)


,ctr1,ctr2,ctr3,trt1,trt2,trt3
gene,,,,,,
Sobic.001G000200,50.804745,51.327542,37.699846,46.737552,37.472610,48.090169
Sobic.001G000400,3.060976,1.382614,3.375644,1.067995,1.233556,1.308627


**TPM (Transcripts per million)**

TPM is proposed as an alternative to RPKM because of inaccuracy in RPKM measurement. 
In contrast to RPKM, the TPM average is constant and is proportional to the relative RNA molar concentration (rmc).


In [13]:
from bioinfokit.analys import norm, get_data
# load sugarcane RNA-seq expression dataset (Published in Bedre et al., 2019)
df = get_data('sc_exp').data
df.head(2)


,gene,ctr1,ctr2,ctr3,trt1,trt2,trt3,length
0,Sobic.001G000200,338,324,246,291,202,168,1982.0
1,Sobic.001G000400,49,21,53,16,16,11,4769.0


In [14]:
# make gene column as index column
df = df.set_index('gene')


In [15]:
# now, normalize raw counts using TPM method
# gene length must be in bp
nm = norm()
nm.tpm(df=df, gl='length')
# get TPM normalized dataframe
tpm_df = nm.tpm_norm
tpm_df.head(2)


,ctr1,ctr2,ctr3,trt1,trt2,trt3
gene,,,,,,
Sobic.001G000200,99.730156,97.641941,72.361658,89.606265,69.447237,90.643338
Sobic.001G000400,6.008723,2.630189,6.479263,2.047584,2.286125,2.466582


**TMM (Trimmed Mean of M-values)**

TMM is a between-sample normalization method in contrast to within-sample normalization methods (RPM, RPKM/FPKM, or TPM)

TMM normalization method assumes that most of the genes are not differentially expressed
TMM normalize the total RNA output among the samples and does not consider gene length or library size for normalization.
TMM considers sample RNA population and effective in normalization of samples with diverse RNA repertoires (e.g. samples from different tissues). TMM will be good choice to remove the batch effects while comparing the samples from different tissues or genotypes or in cases where RNA population would be significantly different among the samples.
To calculate TMM,
get the library size normalized read count for each gene in each sample
calculate the log2 fold change between the two samples (M value)
get absolute expression count (A value)

Now, double trim the upper and lower percentages of the data (trim M values by 30% and A values by 5%)
Get weighted mean of M after trimming and calculate normalization factor.

TMM is implemented in edgeR and performs better for between-samples comparisons
edgeR does not consider gene length for normalization as it assumes that the gene length would be constant between the samples.



In [1]:
# load library
library(edgeR)
x <- read.csv("https://reneshbedre.github.io/assets/posts/gexp/df_sc.csv",row.names="gene")
# delete last column (gene length column)
x <- x[,-7]
head(x)


Loading required package: limma



,ctr1,ctr2,ctr3,trt1,trt2,trt3
,<int>,<int>,<int>,<int>,<int>,<int>
Sobic.001G000200,338,324,246,291,202,168
Sobic.001G000400,49,21,53,16,16,11
Sobic.001G000700,39,49,30,46,52,25
Sobic.001G000800,530,530,499,499,386,264
Sobic.001G001000,12,3,4,3,10,7
Sobic.001G001132,4,2,2,3,4,1


In [2]:
group <- factor(c('c','c', 'c', 't', 't', 't'))
y <- DGEList(counts=x, group=group)
# normalize for library size by cacluating scaling factor using TMM (default method)
y <- calcNormFactors(y)
# normalization factors for each library
y$samples


,group,lib.size,norm.factors
,<fct>,<dbl>,<dbl>
ctr1,c,3356671,1.0289664
ctr2,c,3184864,0.9919562
ctr3,c,3292243,1.0479464
trt1,t,3141401,0.9651375
trt2,t,2719780,0.9819645
trt3,t,1762582,0.9864663


In [3]:
# count per million read (normalized count)
norm_counts <- cpm(y)
head(norm_counts)    


,ctr1,ctr2,ctr3,trt1,trt2,trt3
Sobic.001G000200,97.860339,102.5561297,71.3023988,95.9799323,75.634827,96.6223700
Sobic.001G000400,14.186854,6.6471566,15.3618989,5.2772471,5.990877,6.3264647
Sobic.001G000700,11.291578,15.5100320,8.6954145,15.1720855,19.470352,14.3783289
Sobic.001G000800,153.449643,167.7615701,144.6337277,164.5841451,144.529917,151.8351528
Sobic.001G001000,3.474332,0.9495938,1.1593886,0.9894838,3.744298,4.0259321
Sobic.001G001132,1.158111,0.6330625,0.5796943,0.9894838,1.497719,0.5751332


**DESeq or DESeq2 normalization (median-of-ratios method)**


DESeq2 requires raw counts (not normalized) as integer values for differential expression analysis. If you have expected counts from RSEM, it is recommended to use tximport to import the counts and then to use DESeqDataSetFromTximport() for performing differential expression analysis using DESeq2. In addition, you can also round the expected counts from RSEM but it does not offer the benefits of tximport such as normalization of transcript lengths per gene for gene-level expression analysis.

In [9]:
# load library
library(DESeq2)
x <- read.csv("https://reneshbedre.github.io/assets/posts/gexp/df_sc.csv",row.names="gene")
cond <- read.csv("https://reneshbedre.github.io/assets/posts/gexp/condition.csv",row.names="sample")
cond$condition <- factor(cond$condition)
# keep only required columns present in the sample information table
x <- x[, rownames(cond)]
head(x)


Warning message:
“package ‘DESeq2’ was built under R version 4.3.3”
Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following object is masked from ‘package:limma’:

    plotMA


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min



Attaching package: ‘S4Vectors’


The following object is masked from ‘package:utils’:

    findMatches


The following objects are masked from ‘package:base’:

    expand.gr

,ctr1,ctr2,ctr3,trt1,trt2,trt3
,<int>,<int>,<int>,<int>,<int>,<int>
Sobic.001G000200,338,324,246,291,202,168
Sobic.001G000400,49,21,53,16,16,11
Sobic.001G000700,39,49,30,46,52,25
Sobic.001G000800,530,530,499,499,386,264
Sobic.001G001000,12,3,4,3,10,7
Sobic.001G001132,4,2,2,3,4,1


In [8]:
#BiocManager::install("DESeq2")


'getOption("repos")' replaces Bioconductor standard repositories, see
'help("repositories", package = "BiocManager")' for details.
Replacement repositories:
    CRAN: https://cran.r-project.org

Bioconductor version 3.18 (BiocManager 1.30.23), R 4.3.2 (2023-10-31)

Installing package(s) 'DESeq2'




The downloaded binary packages are in
	/var/folders/3f/6pzn2nyn32d7wthyxkzsqyxcdrtvym/T//Rtmpp6YezD/downloaded_packages


Old packages: 'admisc', 'arm', 'brms', 'broom.mixed', 'clusterProfiler',
  'codetools', 'datawizard', 'DBI', 'dunn.test', 'effectsize', 'emmeans',
  'estimability', 'FactoMineR', 'future', 'future.apply', 'geepack',
  'ggeffects', 'ggfortify', 'ggfun', 'glmmTMB', 'hardhat', 'Hmisc', 'insight',
  'ivreg', 'knitr', 'markdown', 'mclust', 'mlbench', 'mvtnorm', 'parameters',
  'performance', 'QuickJSR', 'RcppArmadillo', 'repr', 'scatterpie',
  'seriation', 'sjmisc', 'sjPlot', 'sjstats', 'SparseM', 'StanHeaders',
  'StatMatch', 'StepReg', 'STRINGdb', 'TMB', 'VGAM', 'WriteXLS', 'xts', 'brio',
  'bslib', 'cachem', 'callr', 'dbplyr', 'digest', 'fastmap', 'fs', 'gh',
  'highr', 'htmltools', 'httpuv', 'httr2', 'KernSmooth', 'lattice', 'nlme',
  'openssl', 'pkgbuild', 'pkgdown', 'processx', 'promises', 'ragg', 'rlang',
  'rmarkdown', 'rstudioapi', 'sass', 'shiny', 'stringi', 'survival',
  'systemfonts', 'testthat', 'textshaping', 'tinytex', 'xfun', 'xopen'



In [10]:
# get dds
dds <- DESeqDataSetFromMatrix(countData = x, colData = cond, design = ~ condition)
dds <- estimateSizeFactors(dds)
# DESeq2 normalization counts
y = counts(dds, normalized = TRUE)
head(y)


,ctr1,ctr2,ctr3,trt1,trt2,trt3
Sobic.001G000200,272.483741,290.412982,199.133348,272.915069,211.917896,271.037655
Sobic.001G000400,39.502081,18.823064,42.902713,15.005640,16.785576,17.746513
Sobic.001G000700,31.440432,43.920482,24.284555,43.141214,54.553122,40.332984
Sobic.001G000800,427.267404,475.058273,403.933092,467.988384,404.952020,425.916314
Sobic.001G001000,9.673979,2.689009,3.237941,2.813557,10.490985,11.293236
Sobic.001G001132,3.224660,1.792673,1.618970,2.813557,4.196394,1.613319


In [11]:
# get size factors
sizeFactors(dds)


ctr1      ctr2      ctr3      trt1      trt2      trt3 
1.2404410 1.1156526 1.2353531 1.0662658 0.9531993 0.6198401

**GeTMM method**


In [12]:
# load library
library(edgeR)
x <- read.csv("https://reneshbedre.github.io/assets/posts/gexp/df_sc.csv",row.names="gene")

# calculate reads per Kbp of gene length (corrected for gene length)
# gene length is in bp in exppression dataset and converted to Kbp
rpk <- ( (x[,1:6]*10^3 )/x[,7])
# comparing groups
group <- factor(c('c','c', 'c', 't', 't', 't'))
y <- DGEList(counts=rpk, group=group)
# normalize for library size by cacluating scaling factor using TMM (default method)
y <- calcNormFactors(y)
# normalization factors for each library
y$samples
# count per million read (normalized count)
norm_counts <- cpm(y)
head(norm_counts)


,group,lib.size,norm.factors
,<fct>,<dbl>,<dbl>
ctr1,c,1709962.4,1.0768758
ctr2,c,1674190.8,0.9843576
ctr3,c,1715232.3,1.0497554
trt1,t,1638517.0,0.9842029
trt2,t,1467549.5,0.9432673
trt3,t,935125.2,0.9679969


,ctr1,ctr2,ctr3,trt1,trt2,trt3
Sobic.001G000200,92.610641,99.193569,68.931922,91.044504,73.624134,93.640111
Sobic.001G000400,5.579774,2.671986,6.172164,2.080449,2.423623,2.548130
Sobic.001G000700,19.324217,27.128619,15.201962,26.026254,34.274038,25.199141
Sobic.001G000800,74.411018,83.144124,71.647825,79.997879,72.089717,75.400417
Sobic.001G001000,9.283077,2.593142,3.164549,2.650016,10.290474,11.015830
Sobic.001G001132,7.464743,4.170414,3.817033,6.392823,9.929777,3.796324


In [ ]:
##Refrence: https://www.reneshbedre.com/blog/expression_units.html